## for prediction of credit default

In [ ]:
pip install catboost

     |████████████████████████████████| 66.1MB 70kB/s 


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score, log_loss
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
import catboost 
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_validate
import imblearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Train1.csv')
df_Test = pd.read_csv('/content/drive/My Drive/Test1.csv')
df_sample = pd.read_csv('/content/drive/My Drive/SampleSubmission1.csv')
df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [ ]:
df_user = df.pop('Applicant_ID')
df_Test_user =df_Test.pop('Applicant_ID')


In [ ]:
# combine both test and train to 
df_Test['default_status']=-1

data = pd.concat((df, df_Test)).reset_index(drop=True)
print(df.shape, df_Test.shape, data.shape)
data.head()

(56000, 51) (24000, 51) (80000, 51)


,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 51 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   form_field1     76361 non-null  float64
 1   form_field2     74447 non-null  float64
 2   form_field3     79499 non-null  float64
 3   form_field4     79499 non-null  float64
 4   form_field5     79499 non-null  float64
 5   form_field6     61036 non-null  float64
 6   form_field7     72606 non-null  float64
 7   form_field8     61036 non-null  float64
 8   form_field9     68592 non-null  float64
 9   form_field10    79498 non-null  float64
 10  form_field11    35181 non-null  float64
 11  form_field12    65922 non-null  float64
 12  form_field13    71648 non-null  float64
 13  form_field14    80000 non-null  int64  
 14  form_field15    47933 non-null  float64
 15  form_field16    61490 non-null  float64
 16  form_field17    64154 non-null  float64
 17  form_field18    65229 non-null 

In [ ]:
data.isna().sum()#checks for number of missing data

form_field1        3639
form_field2        5553
form_field3         501
form_field4         501
form_field5         501
form_field6       18964
form_field7        7394
form_field8       18964
form_field9       11408
form_field10        502
form_field11      44819
form_field12      14078
form_field13       8352
form_field14          0
form_field15      32067
form_field16      18510
form_field17      15846
form_field18      14771
form_field19          4
form_field20        502
form_field21      22561
form_field22      29124
form_field23      40248
form_field24      18902
form_field25       7706
form_field26      10610
form_field27      13209
form_field28        502
form_field29        502
form_field30      36417
form_field31      56218
form_field32       7706
form_field33       1751
form_field34        502
form_field35      33014
form_field36       2898
form_field37       7706
form_field38        502
form_field39       6040
form_field40      62557
form_field41      54578
form_field42    

In [ ]:
#mapping and converting categoricals 
data['form_field47']= data['form_field47'].map({'charge':0, 'lending':1}, na_action= 'ignore')

data['default_status']= data['default_status'].map({'yes':1, 'no':0}, na_action= 'ignore')


In [ ]:
#data.describe()
data['form_field36'].mean()

12.834530881170398

In [ ]:
np.all(np.isfinite(data))#this shows there is infinity

False

In [ ]:
data.columns

Index(['form_field1', 'form_field2', 'form_field3', 'form_field4',
       'form_field5', 'form_field6', 'form_field7', 'form_field8',
       'form_field9', 'form_field10', 'form_field11', 'form_field12',
       'form_field13', 'form_field14', 'form_field15', 'form_field16',
       'form_field17', 'form_field18', 'form_field19', 'form_field20',
       'form_field21', 'form_field22', 'form_field23', 'form_field24',
       'form_field25', 'form_field26', 'form_field27', 'form_field28',
       'form_field29', 'form_field30', 'form_field31', 'form_field32',
       'form_field33', 'form_field34', 'form_field35', 'form_field36',
       'form_field37', 'form_field38', 'form_field39', 'form_field40',
       'form_field41', 'form_field42', 'form_field43', 'form_field44',
       'form_field45', 'form_field46', 'form_field47', 'form_field48',
       'form_field49', 'form_field50', 'default_status'],
      dtype='object')

##  feature Engineering

In [ ]:
data.form_field17=data.form_field17.combine_first(data.form_field18)#it takes values from 18
#print(df.form_field18)
data.form_field17.isna().sum()
#df['form_field17'].fillna(method='backfill', inplace=True, axis=1)#takes values from wherever
#print(df['form_field17'])



14771

In [ ]:
#data['form_field18'].fillna(method='ffill', inplace=True)#takes values from wherever
#print(data['form_field18'])  

data.form_field18=data.form_field18.combine_first(data.form_field17)#it takes values from 17
#print(data.form_field18)
data.form_field18.isna().sum()
#or data.form_field18=data.form_field18.fillna(data.form_field17)
#print(sum(data.form_field18))
#print(data.form_field18)


14771

In [ ]:
data.form_field20=data.form_field20.combine_first(data.form_field19)#it takes values from 19
#print(data_Test.form_field20)
data.form_field20.isna().sum()

#df['form_field20'].fillna(method='ffill', inplace=True, axis=1)# fills from wherever
#print(df['form_field20'])


4

In [ ]:
data.form_field19=data.form_field19.combine_first(data.form_field20)#it takes values from 20
#print(data.form_field18)
data.form_field19.isna().sum()
#['form_field19'].fillna(method='backfill', inplace=True, axis=1)  # fills from wherever
#print(data['form_field19']) or print(data.form_field19)


4

In [ ]:
data.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
count,76361.000000,74447.000000,79499.000000,79499.000000,79499.000000,6.103600e+04,7.260600e+04,6.103600e+04,6.859200e+04,7.949800e+04,3.518100e+04,6.592200e+04,7.164800e+04,8.000000e+04,4.793300e+04,61490.000000,65229.000000,65229.000000,79996.000000,79996.000000,57439.000000,50876.000000,39752.000000,61098.000000,72294.000000,69390.000000,66791.000000,79498.000000,79498.000000,43583.000000,23782.000000,72294.000000,78249.000000,79498.000000,46986.000000,77102.000000,72294.000000,79498.000000,73960.000000,17443.000000,25422.000000,78099.000000,79182.000000,72255.000000,35145.000000,57211.000000,80000.000000,5.018900e+04,79499.000000,6.414700e+04,56000.000000
mean,3491.942169,0.552815,1.056191,0.854130,2.024495,6.250072e+05,6.844769e+06,2.634964e+06,1.326390e+07,1.186153e+07,2.078096e+06,2.771394e+06,6.002145e+06,7.627971e+07,1.143972e+08,0.231582,0.262997,0.272026,0.945697,1.021851,86.685145,6635.784763,6458.894743,104.972751,2625.097954,10107.365543,6193.160306,17406.704445,16515.352235,5223.760549,10880.678959,940.861759,13.993114,0.230094,0.198123,12.834531,8.502725,8.572845,0.090819,147.746178,108.145137,0.368656,6.618305,0.564228,0.070736,0.096135,0.352000,3.041331e+05,1.053579,6.228580e+05,0.244911
std,189.075233,0.822650,2.163093,3.233269,10.791811,1.440721e+06,1.831476e+07,3.939693e+06,2.076362e+07,2.613218e+07,5.174039e+06,4.621145e+06,2.226305e+07,3.068579e+09,1.166052e+08,0.622041,0.709826,0.719861,1.727162,1.773124,53.365748,5673.072482,5797.309660,450.120664,1959.001865,8793.726717,5390.116309,26998.346573,9667.958861,3128.615833,6085.942717,1048.641698,15.300278,0.679609,0.569388,10.571086,9.371225,7.928927,0.419301,44.643599,36.663235,0.413222,6.374198,0.196703,0.287365,0.346109,0.477597,1.717065e+06,1.809372,6.066510e+06,0.430038
min,2986.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.395430e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.255000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166600,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,3358.000000,0.070050,0.000000,0.000000,0.000000,1.400400e+04,6.830840e+05,1.906100e+05,1.360722e+06,4.411260e+05,2.139500e+05,2.746340e+05,1.941110e+05,1.657646e+07,4.994838e+07,0.000000,0.000000,0.000000,0.000000,0.000000,40.395000,1090.072500,811.736250,14.838750,1231.500000,3375.000000,2068.000000,60.000000,9765.000000,2312.000000,6297.000000,194.000000,2.333400,0.000000,0.000000,6.000000,2.000000,4.000000,0.000000,136.033500,87.229500,0.000000,2.020000,0.412176,0.000000,0.000000,0.000000,7.706548e+01,0.000000,4.545456e-02,0.000000
50%,3484.000000,0.269450,0.060000,0.000000,0.000000,1.155330e+05,2.710163e+06,9.639420e+05,5.512519e+06,3.692582e+06,6.924200e+05,1.158248e+06,1.591010e+06,2.814143e+07,8.408313e+07,0.000000,0.000000,0.000000,0.000000,0.000000,89.535000,5961.637500,5550.000000,60.660000,2208.000000,8214.000000,5110.000000,486.000000,15057.000000,5658.000000,10584.000000,600.000000,7.833400,0.000000,0.000000,10.000000,6.000000,6.000000,0.000000,150.000000,118.325250,0.220000,5.050000,0.589624,0.000000,0.000000,0.000000

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = data.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * data.isnull().sum() / len(data)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(data.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns


In [ ]:
# Missing values statistics
missing_values = missing_values_table(data)
missing_values.head(20)

Your selected dataframe has 51 columns.
There are 49 columns that have missing values.


,Missing Values,% of Total Values
form_field40,62557,78.2
form_field31,56218,70.3
form_field41,54578,68.2
form_field45,44855,56.1
form_field11,44819,56.0
form_field23,40248,50.3
form_field30,36417,45.5
form_field35,33014,41.3
form_field15,32067,40.1
form_field48,29811,37.3


In [ ]:
data=data.drop(['form_field40'],axis=1)# at here,with feat.eng with -1=0.8305257. highest so far. MowopeMart1rf-1_ = MowopeMart1rf-1__

In [ ]:
## this replaces Nan with -1

data=data.fillna(-1)

# this replaces Nan with -99999
#data = data.fillna(-99999)


## this replaces NaN with the mean of its column

#column_means = data.mean() 
#data=data.fillna(column_means)


# this replaces Nan with 0

#data=data.fillna(0)


#or 
#data=data.replace([np.inf, -np.inf], np.nan)

# replace NaN with value
#column_means = data.mean()
#data=data.fillna(column_means)

#data=data.fillna(-1)

#data = data.fillna(-99999)

#data=data.fillna(0)


In [ ]:
data['form_field17_19'] = data['form_field17'] + data['form_field19'] 
data =data.drop(['form_field17','form_field19'],axis=1)


In [ ]:
data['form_field18_20'] = (data['form_field18'] + data['form_field20'])
data=data.drop(['form_field18','form_field20'],axis=1)

In [ ]:
data['form_field1719_1820'] = (data['form_field17_19'] + data['form_field18_20'])
data=data.drop(['form_field17_19','form_field18_20'],axis=1)

In [ ]:
data['form_field32_37'] = (data['form_field32'] + data['form_field37'])
data=data.drop(['form_field32','form_field37'],axis=1)

In [ ]:
data['form_field4_46'] = data['form_field4'] + data['form_field46']
data=data.drop(['form_field4','form_field46'],axis=1)

In [ ]:
data['form_field1_28'] = data['form_field1'] + data['form_field28']
data =data.drop(['form_field1','form_field28'],axis=1)

In [ ]:
data['form_field26_27'] = (data['form_field26'] +  data['form_field27'])
data=data.drop(['form_field26','form_field27'],axis=1)

In [ ]:
data['form_field7_10'] = (data['form_field7'] + data['form_field10'])
data=data.drop(['form_field7','form_field10'],axis=1)

In [ ]:
data['form_field25_29'] = (data['form_field25'] + data['form_field29'])
data=data.drop(['form_field25','form_field29'],axis=1)

In [ ]:
data=data.drop(['form_field1719_1820'],axis=1) # at thisnew point,missing is -1 =0.829, 0 as ,0=0.826

In [ ]:
print(data.shape)
data.head(1)

(80000, 40)


,form_field2,form_field3,form_field5,form_field6,form_field8,form_field9,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field21,form_field22,form_field23,form_field24,form_field30,form_field31,form_field33,form_field34,form_field35,form_field36,form_field38,form_field39,form_field41,form_field42,form_field43,form_field44,form_field45,form_field47,form_field48,form_field49,form_field50,default_status,form_field32_37,form_field4_46,form_field1_28,form_field26_27,form_field7_10,form_field25_29
0,0.28505,1.656,0.0,0.0,252072.0,4272776.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,134.655,-1.0,-1.0,134.655,5962.0,-1.0,8.0,2.0,-1.0,12.0,4.0,0.0,-1.0,0.392854,2.02,0.711632,0.0,0,-1.0,1.129518,0.044335,0.0,1064.0,0.0,3496.0,28440.0,22022846.0,21064.5


## model

In [ ]:
X=data[:df.shape[0]]
X.drop('default_status', axis=1, inplace=True)
X = StandardScaler().fit_transform(X)

y = data.default_status[:df.shape[0]].copy()#map({'yes':1, 'no':0}, na_action= 'ignore')


print(len(X),len(y))
print(X.shape[1])#the no of columns
print(y.tail())
#print(X.head())

56000 56000
39
55995    0.0
55996    1.0
55997    0.0
55998    0.0
55999    0.0
Name: default_status, dtype: float64


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x=data[df.shape[0]:].astype(float)
x.drop('default_status', axis=1,inplace=True)
x = StandardScaler().fit_transform(x)
print(len(x))

24000


In [ ]:
#Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 0)


In [ ]:
pip install scikit-optimize

     |████████████████████████████████| 102kB 2.6MB/s 


In [ ]:
pip install shap

     |████████████████████████████████| 327kB 3.3MB/s 
  Created wheel for shap: filename=shap-0.36.0-cp36-cp36m-linux_x86_64.whl size=456470 sha256=f655c4420ebd4f704dc28b1b7b261adb1763b1a5bf3d9279e7f7786d8fb8c067
  Stored in directory: /root/.cache/pip/wheels/fb/15/e1/8f61106790da27e0765aaa6e664550ca2c50ea339099e799f4
Successfully built shap


In [ ]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt.plots import plot_convergence
from copy import deepcopy
import pprint
import shap
pp = pprint.PrettyPrinter(indent=4)
% matplotlib inline

In [ ]:
class ModelOptimizer:
    best_score = None
    opt = None
    
    def __init__(self, model, X_train, y_train, categorical_columns_indices=None, n_fold=3, seed=2405, early_stopping_rounds=30, is_stratified=True, is_shuffle=True):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.categorical_columns_indices = categorical_columns_indices
        self.n_fold = n_fold
        self.seed = seed
        self.early_stopping_rounds = early_stopping_rounds
        self.is_stratified = is_stratified
        self.is_shuffle = is_shuffle
        
        
    def update_model(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self.model, k, v)
            
    def evaluate_model(self):
        pass
    
    def optimize(self, param_space, max_evals=10, n_random_starts=2):
        start_time = time.time()
        
        @use_named_args(param_space)
        def _minimize(**params):
            self.model.set_params(**params)
            return self.evaluate_model()
        
        opt = gp_minimize(_minimize, param_space, n_calls=max_evals, n_random_starts=n_random_starts, random_state=2405, n_jobs=-1)
        best_values = opt.x
        optimal_values = dict(zip([param.name for param in param_space], best_values))
        best_score = opt.fun
        self.best_score = best_score
        self.opt = opt
        
        print('optimal_parameters: {}\noptimal score: {}\noptimization time: {}'.format(optimal_values, best_score, time.time() - start_time))
        print('updating model with optimal values')
        self.update_model(**optimal_values)
        plot_convergence(opt)
        return optimal_values
 
class XgbOptimizer(ModelOptimizer):
    def evaluate_model(self):
        scores = xgboost.cv(self.model.get_xgb_params(), 
                    xgboost.DMatrix(self.X_train, label=self.y_train),
                    num_boost_round=self.model.n_estimators, 
                    metrics='auc', 
                    nfold=self.n_fold, 
                    stratified=self.is_stratified,
                    shuffle=self.is_shuffle,
                    seed=self.seed,
                    early_stopping_rounds=self.early_stopping_rounds)
        self.scores = scores
        test_scores = scores.iloc[:, 2]
        best_metric = test_scores.max()
        return 1 - best_metric
    
class CatboostOptimizer(ModelOptimizer):
    def evaluate_model(self):
        validation_scores = catboost.cv(
        catboost.Pool(self.X_train, 
                      self.y_train, 
                      cat_features=self.categorical_columns_indices),
                      self.model.get_params(), 
                      nfold=self.n_fold,
                      stratified=self.is_stratified,
                      seed=self.seed,
                      early_stopping_rounds=self.early_stopping_rounds,
                      shuffle=self.is_shuffle,
                      plot=False)
                      
        self.scores = validation_scores
        test_scores = validation_scores.iloc[:, 1]
        best_metric = test_scores.max()
        return 1 - best_metric 
    
class LightGBMOptimizer(ModelOptimizer):
    def evaluate_model(self):
        lgb_dataset = lgb.Dataset(self.X_train, 
                                  self.y_train, 
                                  self.categorical_columns_indices)
        eval_hist = lgb.cv(self.model.get_params(), 
                           lgb_dataset,
                           self.model.n_estimators, 
                           nfold=self.n_fold,
                           seed=self.seed, 
                           stratified=self.is_stratified, 
                           shuffle=self.is_shuffle,
                           early_stopping_rounds=self.early_stopping_rounds, 
                           metrics='auc')
        self.scores = eval_hist
        test_scores = eval_hist[list(eval_hist.keys())[0]]
        best_metric = max(test_scores)
        return 1 - best_metric

In [ ]:
# default xgb 
default_xgb = xgboost.XGBClassifier(
        objective='binary:logistic',
        n_jobs=-1
    )

default_xgb_optimizer = XgbOptimizer(default_xgb, X_train, y_train)
default_xgb_optimizer.evaluate_model()

In [ ]:
# greed parameter tuning for xgboost
xgb = xgboost.XGBClassifier(
        n_estimators=10000, # use large n_estimators deliberately to make use of the early stopping
        objective='binary:logistic',
        n_jobs=-1
    )

optimizer = XgbOptimizer(xgb, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate')]
optimal_learning_rate = optimizer.optimize(params_space)

In [ ]:
params_space = [
    Integer(2, 10, name='max_depth'), 
    Integer(1, 20, name='min_child_weight')
]
optimal_values = optimizer.optimize(params_space)

In [ ]:
params_space = [
    Real(0.5, 1.0, name='subsample'), 
    Real(0.5, 1.0, name='colsample_bytree')
]
optimal_values = optimizer.optimize(params_space)

In [ ]:
params_space = [
    Real(1.0, 16.0, name='scale_pos_weight'), 
    Real(0.0, 100, name='reg_lambda')
]
optimal_values = optimizer.optimize(params_space)

In [ ]:
optimizer.model

In [ ]:
# one step optimizer
xgb = xgboost.XGBClassifier(
        n_estimators=10000, # use large n_estimators deliberately to make use of the early stopping
        objective='binary:logistic',
        n_jobs=-1
    )
one_optimizer = XgbOptimizer(xgb, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate'), 
                Integer(2, 10, name='max_depth'), 
                Integer(1, 20, name='min_child_weight'), 
                Real(0.5, 1.0, name='subsample'), 
                Real(0.5, 1.0, name='colsample_bytree'), 
                Real(1.0, 16.0, name='scale_pos_weight'), 
                Real(0.0, 100, name='reg_lambda')]
one_optimal_values = one_optimizer.optimize(params_space, max_evals=40)

In [ ]:
one_optimizer.model

In [ ]:
# analyzing xgb model
xgb = optimizer.model
xgb.fit(X_train, y_train)
xgboost.plot_importance(xgb, title='XGBoost Feature Importance')

In [ ]:
# lgb default params
default_lgbm = lgb.LGBMClassifier(
        objective='binary',
        n_jobs=-1
    )

default_lgbm_optimizer = LightGBMOptimizer(default_lgbm, X_train, y_train)
default_lgbm_optimizer.evaluate_model()

In [ ]:
#greedy parameter tuning
greedy_lgbm = lgb.LGBMClassifier(n_estimators=10000, # use large n_estimators deliberately to make use of the early stopping
        objective='binary',
        n_jobs=-1)
greedy_lgbm_optimizer = LightGBMOptimizer(greedy_lgbm, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate'),]
greedy_lgbm_optimal_values = greedy_lgbm_optimizer.optimize(params_space)

In [ ]:
params_space = [
    Integer(2, 12, name='max_depth'), 
    Integer(1, 20, name='min_child_weight'),
    Integer(10, 50, name='min_child_samples')
]
optimal_values = greedy_lgbm_optimizer.optimize(params_space)

In [ ]:
params_space = [
    Real(0.5, 1.0, name='subsample'), 
    Real(0.5, 1.0, name='colsample_bytree')
]
optimal_values = greedy_lgbm_optimizer.optimize(params_space)

In [ ]:
params_space = [
    Real(1.0, 16.0, name='scale_pos_weight'), 
    Real(0.0, 100, name='reg_lambda')
]
optimal_values = greedy_lgbm_optimizer.optimize(params_space)

In [ ]:
# one time optimizer
lgbm = lgb.LGBMClassifier(n_estimators=10000, # use large n_estimators deliberately to make use of the early stopping
        objective='binary',
        n_jobs=-1)
lgbm_optimizer = LightGBMOptimizer(lgbm, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate'), 
                Integer(2, 12, name='max_depth'), 
                Integer(1, 20, name='min_child_weight'),
                Integer(10, 50, name='min_child_samples'),
                Real(0.5, 1.0, name='subsample'), 
                Real(0.5, 1.0, name='colsample_bytree'), 
                Real(1.0, 16.0, name='scale_pos_weight'), 
                Real(0.0, 100, name='reg_lambda')]
lgbm_optimal_values = lgbm_optimizer.optimize(params_space, max_evals=40, n_random_starts=4)

In [ ]:
# default param for catboost
default_cb = catboost.CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', silent=True)
default_cb_optimizer = CatboostOptimizer(default_cb, X_train, y_train)
default_cb_optimizer.evaluate_model()

In [ ]:
#  greedy parameter tuning
cb = catboost.CatBoostClassifier(n_estimators=4000, # use large n_estimators deliberately to make use of the early stopping
                         one_hot_max_size=2,
                         loss_function='Logloss',
                         eval_metric='AUC',
                         boosting_type='Ordered', # use permutations
                         random_seed=2405, 
                         use_best_model=True,
                         silent=True)
cb_optimizer = CatboostOptimizer(cb, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate'),]
cb_optimal_values = cb_optimizer.optimize(params_space)


In [ ]:
params_space = [Integer(2, 10, name='max_depth'),]
cb_optimal_values = cb_optimizer.optimize(params_space)


In [ ]:
params_space = [
                Real(0.5, 1.0, name='colsample_bylevel'), 
                Real(0.0, 100, name='bagging_temperature'),]
cb_optimal_values = cb_optimizer.optimize(params_space)

In [ ]:
params_space = [
                Real(0.0, 100, name='random_strength'), 
                Real(1.0, 100, name='reg_lambda')]
cb_optimal_values = cb_optimizer.optimize(params_space)

In [ ]:
params_space = [Real(1.0, 16.0, name='scale_pos_weight'),]
cb_optimal_values = cb_optimizer.optimize(params_space)
Real(1.0, 16.0, name='scale_pos_weight'),

In [ ]:
cb_optimizer.model.get_params()

In [ ]:
import time
# one step optimizer
cb = catboost.CatBoostClassifier(n_estimators=4000, # use large n_estimators deliberately to make use of the early stopping
                         one_hot_max_size=2,
                         loss_function='Logloss',
                         eval_metric='AUC',
                         boosting_type='Ordered', # use permutations
                         random_seed=2405, 
                         use_best_model=True,
                         silent=True)
one_cb_optimizer = CatboostOptimizer(cb, X_train, y_train)
params_space = [Real(0.01, 0.8, name='learning_rate'), 
                Integer(2, 10, name='max_depth'), 
                Real(0.5, 1.0, name='colsample_bylevel'), 
                Real(1.0, 16.0, name='scale_pos_weight'), 
                Real(0.0, 100, name='bagging_temperature'), 
                Real(0.0, 100, name='random_strength'), 
                Real(1.0, 100, name='reg_lambda')]
one_cb_optimal_values = one_cb_optimizer.optimize(params_space, max_evals=40, n_random_starts=4)

In [ ]:
one_cb_optimizer.model.get_params()

In [ ]:
# model analysis
# rewrite
ctb = optimizer.model
ctb.fit(X_train, y_train)
catboost.plot_importance(ctb, title='CatBoost Feature Importance')


In [ ]:
# evaluating the model
pred = ctb.predict(X_test)
#predict probabilities. keep only probabilities for positive outcome
pred_proba=ctb.predict_proba(X_test)[:, 1]

#printing the predictions
print(pred)
print(pred_proba)
print(len(pred))

In [ ]:
#calculate scores. 100% correct prediction has auc score of 1
print('Auc on test set: {:.4f}'.format(roc_auc_score(y_test, pred_proba)))


In [ ]:
importances = ctb.feature_importances_
for i,v in enumerate(importances):
    print('Feature: %0d, Score: %.5f' %(i,v))
pyplot.bar([x for x in range(len(importances))], importances)

plt.title('feature importances')
pyplot.show()


In [ ]:
ctb = optimizer.model
ctb.fit(X, y)

In [ ]:
'''for pickle'''
with open ('MowopeMart1tc_t.pickle','wb') as f:
    pickle.dump(ctb, f)

In [ ]:
'''we use this cell while testing on new data after we have written pickle'''
'''to read the pickle'''
pickle_in = open('MowopeMart1tc_t.pickle','rb')
'''we renamed classifier here'''
ctb = pickle.load(pickle_in)

In [ ]:
#df_sol = pd.DataFrame(ctb.predict(x))#converting prediction to a dataframe
df_sol=ctb.predict_proba(x)[:, 1]
print(df_sol)
print(len(df_sol))
#print(df_sol_proba)

In [ ]:
df_sample2=df_sample.copy()
df_sample2['default_status']=df_sol
df_sample2.to_csv('MowopeMart1tc_t.csv', index=False) #same as